In [14]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [15]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_sample_weight
import pickle

In [17]:
df = pd.read_csv('../collecting_data/races.csv', index_col=0)

In [18]:
df.shape

(152, 45)

In [19]:
y = df['position_1']
X = df.drop(['position_1'], axis=1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
from sklearn import preprocessing
st_scaler = preprocessing.StandardScaler()
X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

In [22]:
params={'penalty': ['l1', 'l2'],
        'solver': ['saga', 'lbfgs'],
        'C': np.logspace(-3,1,20)}

logreg = LogisticRegression(class_weight='balanced', max_iter=10000)
logreg_cv = GridSearchCV(logreg, params, scoring='f1_micro')
logreg_cv.fit(X_train, y_train)
logreg_predictions = logreg_cv.best_estimator_.predict(X_test)

In [23]:
logreg_cv.best_score_

0.41558441558441556

In [24]:
f1_score(y_test, logreg_cv.best_estimator_.predict(X_test), average='micro')

0.43478260869565216

In [25]:
params={'criterion': ['gini', 'entropy'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 55, 26)) + [None]}
        
rfc = RandomForestClassifier(class_weight='balanced_subsample')
rfc_cv = GridSearchCV(rfc, params, scoring='f1_micro')
rfc_cv.fit(X_train, y_train)
rfc_predictions = rfc_cv.best_estimator_.predict(X_test)

In [26]:
rfc_cv.best_score_

0.5848484848484848

In [27]:
f1_score(y_test, rfc_cv.best_estimator_.predict(X_test), average='micro')

0.6304347826086957

In [28]:
params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'decision_function_shape' : ['ovo', 'ovr']} 

svc = SVC(class_weight='balanced')
svc_cv = GridSearchCV(svc, params, scoring='f1_micro')
svc_cv.fit(X_train, y_train)
svc_predictions = svc_cv.best_estimator_.predict(X_test)

In [29]:
svc_cv.best_score_

0.5004329004329005

In [30]:
f1_score(y_test, svc_cv.best_estimator_.predict(X_test), average='micro')

0.13043478260869565

In [31]:
params = {'depth': [4,5,6,7,8,9, 10],
          'learning_rate' : [0.01,0.02,0.03,0.04],
          'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]}

cbc = CatBoostClassifier(loss_function='MultiClass')
cbc_cv = GridSearchCV(cbc, params, scoring='f1_micro')
cbc_cv.fit(X_train, y_train)
cbc_cv_predictions = cbc_cv.best_estimator_.predict(X_test)

Выходные данные были обрезаны до нескольких последних строк (5000).
10:	learn: 1.8496875	total: 120ms	remaining: 754ms
11:	learn: 1.8420523	total: 139ms	remaining: 787ms
12:	learn: 1.8322390	total: 160ms	remaining: 825ms
13:	learn: 1.8264822	total: 179ms	remaining: 842ms
14:	learn: 1.8190432	total: 198ms	remaining: 857ms
15:	learn: 1.8123341	total: 220ms	remaining: 880ms
16:	learn: 1.8046393	total: 225ms	remaining: 833ms
17:	learn: 1.7987048	total: 240ms	remaining: 828ms
18:	learn: 1.7924000	total: 256ms	remaining: 823ms
19:	learn: 1.7817509	total: 261ms	remaining: 783ms
20:	learn: 1.7715579	total: 264ms	remaining: 741ms
21:	learn: 1.7626710	total: 272ms	remaining: 717ms
22:	learn: 1.7538418	total: 276ms	remaining: 685ms
23:	learn: 1.7474628	total: 292ms	remaining: 681ms
24:	learn: 1.7402524	total: 308ms	remaining: 678ms
25:	learn: 1.7274808	total: 311ms	remaining: 645ms
26:	learn: 1.7167829	total: 313ms	remaining: 615ms
27:	learn: 1.7101633	total: 329ms	remaining: 611ms
28:	learn: 1.7

In [32]:
cbc_cv.best_score_

0.5848484848484848

In [33]:
f1_score(y_test, cbc_cv.best_estimator_.predict(X_test), average='micro')

0.6521739130434783

In [34]:
params = {
    'max_depth': [2,3,4,5,6,7,8,9,10],
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05],
    'scale_pos_weight' : [1, 10, 25, 50, 75, 99, 100, 1000]}

sample_weights = compute_sample_weight(class_weight='balanced', y=y)

xgb = XGBClassifier(n_jobs = -1, objective='multi:softmax', sample_weight=sample_weights)
xgb_cv = GridSearchCV(xgb, params, scoring='f1_micro')
xgb_cv.fit(X_train, y_train)
xgb_predictions = xgb_cv.best_estimator_.predict(X_test)

In [35]:
xgb_cv.best_score_

0.5757575757575757

In [36]:
f1_score(y_test, xgb_cv.best_estimator_.predict(X_test), average='micro')

0.6521739130434783

In [37]:
xgb_cv.best_params_


{'learning_rate': 0.01,
 'max_depth': 2,
 'n_estimators': 180,
 'scale_pos_weight': 1}

In [38]:
sample_weights = compute_sample_weight(class_weight='balanced', y=y)

xgb_model = XGBClassifier(n_jobs = -1, objective='multi:softmax', sample_weight=sample_weights, learning_rate=0.01,
                    max_depth=2, n_estimators=100, scale_pos_weight=1)
xgb_model.fit(X, y)

XGBClassifier(learning_rate=0.01, max_depth=2, n_jobs=-1,
              objective='multi:softprob',
              sample_weight=array([ 0.18765432,  0.18765432,  0.18765432,  0.18765432,  1.9       ,
        0.18765432,  1.9       ,  1.9       ,  0.18765432,  0.18765432,
        0.18765432,  0.18765432,  0.18765432,  0.18765432,  0.18765432,
        1.08571429,  0.18765432,  0.18765432,  0.18765432,  0.18765432,
        1.08571429,  0.18765432,  0.18765432,  1.08571429,  0.18...
        5.06666667,  0.54285714,  0.18765432,  0.54285714,  0.18765432,
        0.18765432,  0.54285714,  5.06666667,  0.54285714,  0.54285714,
        0.54285714,  0.18765432, 15.2       ,  0.54285714,  0.54285714,
        1.9       ,  0.18765432,  1.52      ,  0.54285714,  0.54285714,
        0.18765432,  0.18765432,  0.18765432,  0.54285714,  3.04      ,
        0.54285714,  3.04      ,  0.54285714,  0.54285714,  0.54285714,
        5.06666667,  0.54285714,  0.54285714, 15.2       ,  3.04      ,
        0.54

In [41]:
filename = 'model_2.sav'
pickle.dump(xgb_model, open(filename, 'wb'))